In [1]:
import numpy as np

In [2]:
'''*** Solve lower triangular matrix ***'''
def lower_triangular_solve(A, b):
    """
    Solve the system  A x = b  where A is assumed to be lower triangular,
    i.e. A(i,j) = 0 for j > i, and the diagonal is assumed to be nonzero,
    i.e. A(i,i) != 0.
    
    The code checks that A is lower triangular and converts A and b to
    double precision before computing.

    ARGUMENTS:  A   lower triangular n x n array
                b   right hand side column n-vector

    RETURNS:    x   column n-vector solution
    """

    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')
    
    # checks whether A is lower triangular
    for i in range(n):
        for j in range(i+1,n):
            if not np.isclose(A[i, j], 0.0):
                raise ValueError(f'A is not lower triangular! {A[i, j]=}')

    # checks whether A has zero diagonal element
    for i in range(n):
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A[{i}, {i}] is zero')
    
    # create a new array to store the results
    x = np.empty_like(b)
    
    # perform forward substitution
    x[0] = b[0] / A[0, 0]
    for i in range(1,n):
        x[i] = b[i] / A[i, i]
        for j in range(i):
            x[i] = x[i] - A[i,j]*x[j]/A[i, i]
        
    return x

In [4]:
'''*** Solve upper triangular matrix ***'''
def upper_triangular_solve(A, b):
    """
    Solve the system  A x = b  where A is assumed to be lower triangular,
    i.e. A(i,j) = 0 for j > i, and the diagonal is assumed to be nonzero,
    i.e. A(i,i) != 0.
    
    The code checks that A is lower triangular and converts A and b to
    double precision before computing.

    ARGUMENTS:  A   lower triangular n x n array
                b   right hand side column n-vector

    RETURNS:    x   column n-vector solution
    """

    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
      
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')
    
    # check A is upper triangular
    for i in range(n):
        for j in range(0,i):
            if not np.isclose(A[i, j], 0.0):
                raise ValueError(f'A is not upper triangular! {A[i, j]=}')

    # checks whether A has zero diagonal element
    for i in range(n):
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A[{i}, {i}] is zero')
    
    #create a new array to store the results
    x = np.empty_like(b)
    
    # perform backwards substitution
    x[n-1] = b[n-1] / A[n-1, n-1]
    for i in range(2,n+1):
        x[n-i] = b[n-i] / A[n-i, n-i]
        for j in range(n-i+1, n):
            x[n-i] = x[n-i] - A[n-i,j]*x[j] / A[n-i, n-i]
        
    return x

In [3]:
'''*** Guassian Elimination ***'''
def gaussian_elimination(A, b, verbose=False):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # size of solution vector / the square matrix A
    n=len(b) # or   n, n = A.shape
        
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')
    
    if verbose:
        print('starting system\n', A, b)

    # perform forward elimination
    for i in range(n):  
        # eliminate column i
        if verbose:
            print(f'eliminating column {i}')
        
        # check diagonal
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A has zero on diagonal! A[{i}, {i}] = 0')

        # row j <- row j - (a_{ji} / a_{ii}) row i
        for j in range(i+1, n):
            if verbose:
                print(f'row {j} <- row {j} - {A[j, i] / A[i, i]} row {i}')
            factor = A[j, i] / A[i, i]
            for k in range(0, n):
                A[j, k] = A[j, k] - factor * A[i, k]
            b[j] = b[j] - factor * b[i]
        
        if verbose:
            print('new system\n', A, b)

    return upper_triangular_solve(A, b)   

In [8]:
'''testing'''
U = np.array([[2, 1, 4], [0, 1.5, 0], [0, 0, 2]], dtype=np.double)
b = np.array([[12], [3], [4]], dtype=np.double)

# numpy linear solvers
x0 = np.linalg.solve(U,b)
print(x0)

x = upper_triangular_solve(U, b)
print(x)

np.testing.assert_almost_equal(x, x0)

[[1.]
 [2.]
 [2.]]
[[1.]
 [2.]
 [2.]]


In [9]:
'''testing2'''
A = np.array([[4, 3, 2, 1], [1, 2, 2, 2], [1, 1, 3, 0], [2, 1, 2, 3]], dtype=np.double)
b = np.array([10, 7, 5, 8], dtype=np.double)

# numpy linear solvers
x0 = np.linalg.solve(A,b)
print("x0=", x0)

x = gaussian_elimination(A, b, verbose=False)
print("x=",x)

print(np.matmul(A,x)-b)

# test solution is close to exact value
np.testing.assert_almost_equal(x, x0)
# test residual is small
np.testing.assert_almost_equal(np.matmul(A, x) - b, np.zeros_like(x))

x0= [1. 1. 1. 1.]
x= [1. 1. 1. 1.]
[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 -8.8817842e-16]
